In [1]:
# %matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import numpy as np
from sympy import integrate, Piecewise, symbols, lambdify
import sympy


plt.rc('text', usetex=True)


def plot_analytical(ax, x_vec, sym_func, title="", filename="foo", maxmin_hline=False, x_units="m", y_units='kN \cdot m', ylabel="Bøyemoment", facecolor=None):
    x = symbols('x')
    lambda_function = lambdify(x, sym_func, "numpy")
    y_vec = lambda_function(x_vec)
    return plot_numerical(ax, x_vec, y_vec, title, filename, maxmin_hline, x_units, y_units, ylabel, facecolor)


def plot_numerical(ax, x_vec, y_vec, title="", filename="foo", maxmin_hline=False, x_units="m", y_units='kN \cdot m', ylabel="Bøyemoment", facecolor=None):
    ax.plot(x_vec, y_vec, '0.5', linewidth=2)
    a, b = x_vec[0], x_vec[-1]
    verts = [(a, 0)] + list(zip(x_vec, y_vec)) + [(b, 0)]
    if not facecolor:
        facecolor = '0.9'
    poly = Polygon(verts, facecolor=facecolor, edgecolor='0.5', alpha=0.5)
    ax.add_patch(poly)
    if title:
        ax.set_title(title)
    ax.set_xlim([x_vec.min(), x_vec.max()])
    if maxmin_hline:
        ax.axhline(y=max(y_vec), linestyle='--', color="g", alpha=0.5)
        max_idx = y_vec.argmax()
        ax.axhline(y=min(y_vec), linestyle='--', color="g", alpha=0.5)
        min_idx = y_vec.argmin()
        
        plt.annotate('${:0.1f}'.format(y_vec[max_idx]).rstrip('0').rstrip('.') + " {}$".format(y_units),
                     xy=(x_vec[max_idx], y_vec[max_idx]), xytext=(8, 0), xycoords=('data', 'data'),
                     textcoords='offset points', size=12)
        plt.annotate('${:0.1f}'.format(y_vec[min_idx]).rstrip('0').rstrip('.') + " {}$".format(y_units),
                     xy=(x_vec[min_idx], y_vec[min_idx]), xytext=(8, 0), xycoords=('data', 'data'),
                     textcoords='offset points', size=12)
        ax.set_xlabel('Bjelkeakse $[{}]$'.format(x_units))
        ax.set_ylabel("{} $[{}]$".format(ylabel, y_units))

        plt.savefig('{}.pdf'.format(filename), bbox_inches='tight', rasterized=True)
    return plt

# Test
Reproduction of the example found [here](https://www.mathalino.com/reviewer/mechanics-and-strength-of-materials/solution-to-problem-448-relationship-between-load-shear)

In [59]:
x0, x1 = 0, 9
x_axis = np.linspace(x0, x1, (x1-x0)*1000+1)
filename = "foo"
x = symbols("x")

dist_load = 0
dist_load += Piecewise((-20, x < 2), (0, True))
dist_load += Piecewise((-10, x > 3), (0, True))

shear_force = integrate(dist_load)
point_load_x = 3
point_load_kN = -20
shear_force += Piecewise((0, x<point_load_x), (point_load_kN, True))


# -------- Calculate reaction forces at supports -------- #
F_Rx = 0
F_Ry = integrate(dist_load, (x, x0, x1)) + point_load_kN
M_R = integrate(dist_load * x, (x, x0, x1)) + point_load_kN * point_load_x
print("F_R:\t{0}\nM_R:\t{1}".format(F_R, M_R))

xA, xB = 2, 7
A = np.array([[-1, 0, 0],
              [0, -1, -xA],
              [0, -1, -xB]]).T
b = np.array([F_Rx, F_Ry, M_R])
F_Ax, F_Ay, F_By = np.linalg.inv(A).dot(b)
print("F_Ax:\t{0}\nF_Ay:\t{1}\nF_By:\t{2}".format(F_Ax, F_Ay, F_By))
# -------- Calculate reaction forces at supports -------- #

shear_force += Piecewise((0, x<xA), (F_Ay, True))
shear_force += Piecewise((0, x<xB), (F_By, True))

bending_moment = integrate(shear_force)


fig = plt.figure(figsize=(6, 10))
fig.subplots_adjust(hspace=0.4)

ax1 = fig.add_subplot(3, 1, 1)
plot01_params = {'filename':"foo01", 
               'maxmin_hline':True, 
               'x_units':"m", 
               'y_units':'kN / m', 
               'ylabel':"Distributed loads", 
               'facecolor':"b", 
               'title':"LATEX TEST $- \\frac{2 \\sqrt{11}}{5} + \\frac{23}{5}$"}
plot_analytical(ax1, x_axis, dist_load, **plot01_params);

ax2 = fig.add_subplot(3, 1, 2)
plot02_params = {'filename':"foo02", 
                 'maxmin_hline':True, 
                 'x_units':"m", 
                 'y_units':'kN', 
                 'ylabel':"Shear force", 
                 'facecolor':"r"}
plot_analytical(ax2, x_axis, shear_force, **plot02_params);

ax3 = fig.add_subplot(3, 1, 3)
plot03_params = {'filename':"foo03", 
                 'maxmin_hline':True, 
                 'x_units':"m", 
                 'y_units':'kN \cdot m', 
                 'ylabel':"Bending moment", 
                 'facecolor':"y"}
plot_analytical(ax3, x_axis, bending_moment, **plot03_params);
plt.savefig(fname="test01.pdf")
plt.close()

F_R:	-120
M_R:	-460
F_Ax:	0
F_Ay:	76.0000000000000
F_By:	44.0000000000000
